In [ ]:
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils import *
import concurrent.futures, glob, requests
import os
from os.path import *
from time import perf_counter
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'
import tensorflow as tf
import cv2

imageURLS =['https://esahubble.org/images/heic2017a/',
            'https://esahubble.org/images/heic1917a/',
            'https://esahubble.org/images/opo9828c/',
            'https://esahubble.org/images/potw1714a/',
            'https://esahubble.org/images/opo2059a/',
            'https://esahubble.org/images/opo0217a/',
            'https://esahubble.org/images/heic1815b/'
           ]

downloadCount=0
# for image in sorted(imageURLS):
def downloadImages(imageURLS):
    img_bytes = requests.get(imageURLS).content
    img_name = imageURLS.split('/')[4]
    img_name = f'{img_name}.png'
    
    with open(img_name, 'wb') as image_file:
        image_file.write(img_bytes)
        return f'{C.BIBlue}{img_name}{C.ColorOff} was downloaded'

startTime=perf_counter()
with concurrent.futures.ThreadPoolExecutor() as executor:
    returns = executor.map(downloadImages, imageURLS)
finishTime=perf_counter()
                 
totalTime = round(finishTime - startTime, 0)
print(f'download time: {C.BIBlue}{totalTime}{C.ColorOff}')
for retrn in returns:
    print(retrn)

In [ ]:

def resizeImage(thisPath):
    print(f'{C.BIPurple}{thisPath}{C.ColorOff}')
    img = cv2.imread(thisPath)
    img = tf.image.resize(
        img, size=(224,224), method=tf.image.ResizeMethod.BILINEAR,
        preserve_aspect_ratio=False,
        antialias=False, name=None)
    
    img=np.copy(img)
    cv2.imwrite(thisPath, img)
    sleep(0.2)
    print(img.shape)
    return img

In [ ]:
planetGlobList=[]
planetGlobList=glob.glob('*.png')

for pth in sorted(planetGlobList):
    fullPath=abspath(pth)
    # print(fullPath)
    # img = cv2.imread(fullPath, cv2.IMREAD_COLOR)
    try: img = resizeImage(fullPath)
    except Exception as err: print(err)
    cvu.plotShowSingleImage(img, title1=basename(fullPath))
    